In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-l8ctqev0
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-l8ctqev0
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=28971cfd987948929d051869fd309c297f8cecdc402e3e47d190e720653a5b44
  Stored in directory: /tmp/pip-ephem-wheel-cache-yughoj04/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


### Assignment A1:  Implement Parallel Reduction using Min, Max, Sum and Average operations. 

In [ ]:
%%cu

        
#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>
using namespace std;


__global__ void minimum(int *input)
{
	int tid=threadIdx.x;
	auto step_size=1;
  int number_of_threads=blockDim.x;
  
  while(number_of_threads>0)
  {
      if(tid<number_of_threads)
      {
          int first=tid*step_size*2;
          int second=first+step_size;
          if(input[second]<input[first])
            input[first]=input[second];
      }
      step_size=step_size*2;
      number_of_threads/=2;
  }

}

__global__ void max(int *input)
{
   int tid=threadIdx.x;
   auto step_size=1;
   int number_of_threads=blockDim.x;
   
   while(number_of_threads>0)
   {
       if(tid<number_of_threads)
       {
           int first=tid*step_size*2;
           int second=first+step_size;
           if(input[second]>input[first])
            input[first]=input[second];
       }
       step_size*=2;
       number_of_threads/=2;
   }
}

__global__ void sum(int *input)
{
    const int tid=threadIdx.x;
    auto  step_size=1;
    int number_of_threads=blockDim.x;
    while(number_of_threads>0)
    {
        if(tid<number_of_threads)
        {
            const int first=tid*step_size*2;
            const int second=first+step_size;
            input[first]=input[first]+input[second];
        }
    step_size = step_size*2;; 
		number_of_threads =number_of_threads/2;
    }
}

__global__ void average(int *input)
{
    const int tid=threadIdx.x;
    auto  step_size=1;
    int number_of_threads=blockDim.x;
    int totalElements=number_of_threads*2;
    while(number_of_threads>0)
    {
        if(tid<number_of_threads)
        {
            const int first=tid*step_size*2;
            const int second=first+step_size;
            input[first]=input[first]+input[second];
        }
        step_size = step_size*2;; 
		number_of_threads =number_of_threads/2;
    }
    input[0]=input[0]/totalElements;
}

int main()
{
	int n;
	n=100;
  cout<<"No of Elements: "<<n<<endl;
  srand(n);
	int *arr = new int[n];
	for(int i=0;i<n;i++)
	{
		arr[i]=rand()%20000;
	}

	int size=n*sizeof(int); //calculate no. of bytes for array
	int *arr_d,result1;
	
  //#max operation
  int *arr_max,maxValue;
  cudaMalloc(&arr_max,size);
	cudaMemcpy(arr_max,arr,size,cudaMemcpyHostToDevice);
  //#calling kernel max function
  max<<<1,n/2>>>(arr_max);
	cudaMemcpy(&maxValue,arr_max,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Maximun: "<<maxValue<<endl;

  //#min Operation
	cudaMalloc(&arr_d,size);
	cudaMemcpy(arr_d,arr,size,cudaMemcpyHostToDevice);
  //#calling minimum kernel function
  minimum<<<1,n/2>>>(arr_d);
	cudaMemcpy(&result1,arr_d,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Minimum: "<<result1<<endl;
    
  //#sum operation
  int *arr_sum,sumValue;
  cudaMalloc(&arr_sum,size);
	cudaMemcpy(arr_sum,arr,size,cudaMemcpyHostToDevice);
 //#calling kernal function
  sum<<<1,n/2>>>(arr_sum);
	cudaMemcpy(&sumValue,arr_sum,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Sum: "<<sumValue<<endl; 
  //cout<<"Average:  "<<(sumValue/n)<<endl; 
   
  //#avg operation
  int *arr_avg,avgValue;
  cudaMalloc(&arr_avg,size);
	cudaMemcpy(arr_avg,arr,size,cudaMemcpyHostToDevice);
  //#calling kernal function
  average<<<1,n/2>>>(arr_avg);
	cudaMemcpy(&avgValue,arr_avg,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Average: "<<avgValue<<endl; 
  
   
  //# Free all allcated device memeory
   cudaFree(arr_d);
   cudaFree(arr_sum);
   cudaFree(arr_max);
   cudaFree(arr_avg);
  
return 0;

}

No of Elements: 100
Maximun: 19936
Minimum: 180
Sum: 633114
Average: 6331



### Assignment No. A2 - Vector and Matrix Operations.

In [ ]:
%%cu

#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>

#define n (2048*2048)
#define THREADS_PER_BLOCK 512

using namespace std;

__global__ void add(int *a, int *b, int *c) {
	  int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < n){
        c[index] = a[index] + b[index];
    }
}

int main(){
  //#Addition of two large vectors
  cout<<"No of Elements: "<<n<<endl;
  srand(time(0));
	int *h_firstArr = new int[n];
  int *h_secondArr = new int[n];
  int *h_resultArr = new int[n];

  int *d_firstArr;
  int *d_secondArr;
  int *d_resultArr;

	for(int i=0;i<n;i++)
	{
		h_firstArr[i]=rand()%20000;
		h_secondArr[i]=rand()%1000;
	}


	int size=n*sizeof(int);

	cudaMalloc((void **)&d_firstArr, size);
	cudaMalloc((void **)&d_secondArr, size);
	cudaMalloc((void **)&d_resultArr, size);

	//#Copy inputs to device
	cudaMemcpy(d_firstArr, h_firstArr, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_secondArr, h_secondArr, size, cudaMemcpyHostToDevice);

	add<<<n/THREADS_PER_BLOCK,THREADS_PER_BLOCK>>>(d_firstArr, d_secondArr, d_resultArr);
	//#Copy result back to host
	cudaMemcpy(h_resultArr, d_resultArr, size, cudaMemcpyDeviceToHost);
  cout<<"1. Addition two large vectors: "<<endl;
	for(int i=0; i<100 ;i++){
		cout<<"{"<<i<<"}"<<h_firstArr[i]<<" + "<<h_secondArr[i]<<" = "<<h_resultArr[i]<<" ";
	}

	// Cleanup
	free(h_firstArr); 
	free(h_secondArr); 
	free(h_resultArr);
 
	cudaFree(d_firstArr); 
	cudaFree(d_secondArr); 
	cudaFree(d_resultArr);

	return 0;
}

No of Elements: 4194304
1. Addition two large vectors: 
{0}8034 + 285 = 8319 {1}1305 + 315 = 1620 {2}1449 + 600 = 2049 {3}11660 + 519 = 12179 {4}18553 + 911 = 19464 {5}5561 + 798 = 6359 {6}4877 + 543 = 5420 {7}18283 + 275 = 18558 {8}1995 + 127 = 2122 {9}7343 + 541 = 7884 {10}11439 + 901 = 12340 {11}11727 + 647 = 12374 {12}11782 + 869 = 12651 {13}12990 + 825 = 13815 {14}18464 + 480 = 18944 {15}17532 + 850 = 18382 {16}8117 + 837 = 8954 {17}4518 + 918 = 5436 {18}8790 + 178 = 8968 {19}15437 + 343 = 15780 {20}4089 + 998 = 5087 {21}17493 + 319 = 17812 {22}18893 + 128 = 19021 {23}10946 + 240 = 11186 {24}8608 + 290 = 8898 {25}16781 + 47 = 16828 {26}9543 + 860 = 10403 {27}15046 + 325 = 15371 {28}10729 + 36 = 10765 {29}4502 + 545 = 5047 {30}13868 + 386 = 14254 {31}4748 + 986 = 5734 {32}13576 + 266 = 13842 {33}7904 + 366 = 8270 {34}5444 + 694 = 6138 {35}6061 + 886 = 6947 {36}17044 + 906 = 17950 {37}7557 + 938 = 8495 {38}8386 + 503 = 8889 {39}13178 + 994 = 14172 {40}13145 + 312 = 13457 {41}13393 +

In [ ]:
%%cu

#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>

#define col1row2 2048
#define col2 512

using namespace std;

__global__ void matproduct(int *a,int *b, int *c){
    int x = blockIdx.x;
    int y = blockIdx.y;
    int k;

    c[col2*y+x] = 0;
    for(k = 0; k < col1row2 ; k++){
       c[col2*y+x]= c[col2*y+x]+a[col1row2*y+k]*b[col2*k+x];
    } 

}

int main(){
  //#Multiply vector and matrix
  int *h_vector;
  int *h_resultVector;
  int *h_matrix;
  
  int *d_vector;
  int *d_matrix;
  int *d_resultVector;

  h_vector = (int *)malloc(col1row2*sizeof(int));
  h_resultVector = (int *)malloc(col2*sizeof(int));
  h_matrix =  (int *)malloc(col1row2*col2*sizeof(int));

  cudaMalloc((void **)&d_vector, col1row2*sizeof(int));
	cudaMalloc((void **)&d_matrix, col1row2*col2*sizeof(int));
	cudaMalloc((void **)&d_resultVector, col2*sizeof(int));

  srand(time(0));
  for(int i=0; i < col1row2 ; i++){
     for(int j = 0; j < col2 ; j++){
          h_matrix[i*col2+j] = rand() % 4 + 1;
     }
    h_vector[i]=rand()% 4 + 1;
  } 

  //#Copy inputs to device
	cudaMemcpy(d_vector, h_vector, col1row2*sizeof(int), cudaMemcpyHostToDevice);
	cudaMemcpy(d_matrix, h_matrix, col1row2*col2*sizeof(int), cudaMemcpyHostToDevice);

  dim3 grid(col2,1);

  matproduct<<<grid,1>>>(d_vector,d_matrix,d_resultVector);

 	//#Copy result back to host
	cudaMemcpy(h_resultVector, d_resultVector, col2*sizeof(int), cudaMemcpyDeviceToHost);

  cout << "2. Multiply Vector and Matrix:"<<endl;
  cout << "Vector: "<<endl;
  for(int i=0; i < col1row2 ; i++){
      cout<<h_vector[i]<<" ";
  }
  cout<<endl;
  cout << "Matrix: "<<endl;
   for(int i=0; i < col1row2 ; i++){
     cout <<"{";
     for(int j = 0; j < col2 ; j++){
          cout << h_matrix[i*col2+j]<<" ";
     }
    cout <<"},";
    }

  cout << endl << "Result Vector: "<<endl;
    for(int i=0; i < col2 ; i++){
      cout<<h_resultVector[i]<<" ";
    } 


  //#cleanup
   free(h_vector);
   free(h_resultVector);
   free(h_matrix);

   cudaFree(d_vector);
   cudaFree(d_resultVector);
   cudaFree(d_matrix);
}

2. Multiply Vector and Matrix:
Vector: 
2 3 1 3 1 4 2 3 3 2 2 4 1 4 3 3 1 4 3 1 3 1 1 1 2 2 3 1 3 3 1 3 3 1 1 2 1 1 2 2 2 3 3 4 3 3 4 2 1 1 1 3 4 4 1 2 3 2 4 4 2 2 4 2 4 4 1 1 1 1 2 3 4 4 1 4 4 4 4 1 3 2 4 2 3 1 3 2 4 4 1 2 2 2 1 1 3 2 1 3 2 4 2 2 3 2 4 2 2 3 2 1 4 4 4 2 4 1 3 1 2 4 1 4 2 1 3 1 2 2 1 1 3 3 4 4 2 4 3 3 3 2 1 3 1 3 3 3 1 4 2 4 1 1 4 3 1 2 3 1 3 2 1 3 3 2 4 2 2 2 1 4 1 3 3 3 4 4 3 3 1 3 2 3 3 1 2 1 2 1 4 4 1 3 3 3 3 1 1 2 3 3 3 4 3 2 2 4 3 2 4 2 2 2 3 3 3 1 1 3 1 3 1 3 3 2 3 3 4 3 1 3 1 1 4 3 2 1 1 1 4 3 3 4 3 4 4 1 3 1 1 1 4 3 3 3 3 4 4 2 2 4 2 1 1 4 2 2 2 1 1 2 1 3 2 4 3 2 1 3 1 2 1 1 1 4 2 2 4 1 2 2 2 1 3 2 4 3 4 2 2 4 4 3 4 2 2 1 3 2 3 1 4 2 2 1 3 3 1 3 2 1 4 1 1 1 2 1 4 1 3 3 2 1 4 4 3 4 1 1 2 2 3 2 3 2 3 3 4 1 3 2 3 1 3 4 4 3 1 1 3 1 1 4 3 1 1 4 1 2 2 2 4 2 1 1 4 3 4 3 4 2 3 3 3 3 4 4 3 2 2 1 2 2 3 4 3 1 3 1 3 3 4 1 2 1 3 1 2 3 4 2 1 3 4 4 3 1 3 3 1 4 1 4 3 1 1 3 1 2 2 2 3 3 3 1 4 3 1 2 1 3 3 4 2 1 3 2 4 3 1 1 2 4 4 2 4 2 4 4 3 4 1 3 1 3 4 1 2 3 4 1 3 2 1 3 3 3 1 4 

In [25]:
%%cu

#include<iostream>
#include<cstdlib>
#include<cmath>
#define n 20

using namespace std;

__global__ void matrixMultiplication(int *a, int *b, int *c)
{
    int row = threadIdx.y+blockDim.y*blockIdx.y;
    int col = threadIdx.x+blockDim.x*blockIdx.x;
    int sum = 0;
    //#n[col2*y+x]=n[col2*y+x]+l[col1*y+k]*m[col2*k+x];
    c[n*row+col]=0;
    if(row < n && col < n)
    for(int j=0;j < n;j++)
    {
        c[n*row+col] = c[n*row+col] + a[n*row+j]*b[n*j+col];
    }
    
   //# c[n*row+col]=sum;
}

int main(){
    //#Multiply NxN Arrays
  int *h_matrixA;
  int *h_matrixB;
  int *h_resultMatrixC;
  
  int *d_matrixA;
  int *d_matrixB;
  int *d_resultMatrixC;

  h_matrixA = (int *)malloc(n*n*sizeof(int));
  h_matrixB = (int *)malloc(n*n*sizeof(int));
  h_resultMatrixC =  (int *)malloc(n*n*sizeof(int));

  cudaMalloc((void **)&d_matrixA, n*n*sizeof(int));
	cudaMalloc((void **)&d_matrixB, n*n*sizeof(int));
	cudaMalloc((void **)&d_resultMatrixC, n*n*sizeof(int));

  srand(time(0));
  for(int i=0; i < n ; i++){
     for(int j = 0; j < n ; j++){
          h_matrixA[i*n+j] = rand() % 4 + 1;
          h_matrixB[i*n+j] = rand() % 4 + 1;
     }
  }

  cudaMemcpy(d_matrixA, h_matrixA, n*n*sizeof(int), cudaMemcpyHostToDevice);
	cudaMemcpy(d_matrixB, h_matrixB, n*n*sizeof(int), cudaMemcpyHostToDevice);

  dim3 threadsPerBlock(n, n);
  dim3 blocksPerGrid(1, 1);
    
    if(n*n>512){
        threadsPerBlock.x=512;
        threadsPerBlock.y=512;
        blocksPerGrid.x=ceil((double)n/(double)threadsPerBlock.x);
        blocksPerGrid.y=ceil((double)n/(double)threadsPerBlock.y);
    }

    matrixMultiplication <<<blocksPerGrid,threadsPerBlock>>>(d_matrixA,d_matrixB,d_resultMatrixC);

    //#Copy result back to host
	  cudaMemcpy(h_resultMatrixC, d_resultMatrixC, n*n*sizeof(int), cudaMemcpyDeviceToHost);

   cout << "3. Multiplication of NxN Arrays using n^2 Processors "<<endl;
   cout << "Matrix A: "<<endl;
   for(int i=0; i < n ; i++){
     cout <<"{";
     for(int j = 0; j < n ; j++){
          cout << h_matrixA[i*n+j]<<" ";
     }
    cout <<"},";
    }
    cout<<endl;
    cout << "Matrix B: "<<endl;
    for(int i=0; i < n ; i++){
     cout <<"{";
     for(int j = 0; j < n ; j++){
          cout << h_matrixB[i*n+j]<<" ";
     }
    cout <<"},";
    }
    cout<<endl;
    cout << "Result Matrix C: "<<endl;
    for(int i=0; i < n ; i++){
     cout <<"{";
     for(int j = 0; j < n ; j++){
          cout << h_resultMatrixC[i*n+j]<<" ";
     }
      cout <<"},";
    }

     //#cleanup
   free(h_matrixA);
   free(h_matrixB);
   free(h_resultMatrixC);

   cudaFree(d_matrixA);
   cudaFree(d_matrixB);
   cudaFree(d_resultMatrixC);

}


3. Multiplication of NxN Arrays using n^2 Processors 
Matrix A: 
{4 3 4 3 4 2 2 1 2 2 2 3 4 1 2 2 3 3 3 4 },{4 2 4 2 3 2 3 3 1 1 2 3 2 2 3 1 2 3 4 2 },{3 2 4 3 4 1 1 1 1 1 3 4 3 1 2 3 1 1 1 3 },{2 4 3 4 3 1 1 1 2 4 2 3 3 2 2 2 4 1 1 4 },{1 4 3 4 4 4 1 3 1 1 1 4 3 1 1 2 3 1 4 1 },{3 3 2 2 2 3 4 1 2 4 4 2 1 2 2 2 2 3 2 4 },{1 4 4 1 4 3 1 4 2 1 3 3 3 3 4 2 4 4 4 4 },{1 1 1 2 4 2 1 3 1 2 3 4 3 1 4 4 1 4 3 4 },{4 4 4 2 3 4 1 3 2 4 3 4 2 1 1 1 4 3 2 3 },{2 1 4 4 2 2 1 4 1 1 3 4 1 4 1 2 3 3 1 4 },{4 2 2 1 4 2 1 3 2 4 4 3 3 1 2 4 4 4 2 3 },{4 3 1 2 4 4 1 1 3 1 4 1 2 1 4 4 3 4 4 2 },{1 1 3 4 2 2 4 1 2 4 3 1 1 1 4 2 2 4 4 2 },{3 2 4 3 2 3 4 2 2 4 3 4 4 4 2 3 3 1 2 4 },{4 4 4 4 2 1 4 3 2 1 3 2 4 4 1 4 4 4 3 3 },{1 3 2 3 4 2 3 3 1 2 4 2 4 1 3 2 1 3 2 1 },{1 2 4 4 2 2 2 4 3 4 3 3 1 1 3 2 2 1 4 2 },{4 2 1 3 2 3 2 3 2 4 4 2 1 3 1 4 2 2 3 4 },{1 3 2 4 1 3 1 1 2 4 2 3 4 4 3 1 4 4 1 1 },{3 4 3 3 3 3 4 3 4 1 1 4 1 2 2 1 4 2 4 3 },
Matrix B: 
{1 2 3 2 3 1 4 1 2 1 1 3 1 3 3 2 4 2 2 2 },{1 2 2 2 3 1 1 3 4 2